# **Reto Calles abarrotadas con Enfoque de Sistema de Hormigas**

* Santiago Castro Benavides 			A01799544
* Rodrigo Lira Del Ángel 			A01799277
* Diego Jesús de Lara de la Cruz		A01799544
* Andrés Felipe García Viña                              A01800027

## **Contextualización**

En las grandes ciudades, la creciente demanda de compras en línea ha sobrecargado los sistemas de entrega, llevando a situaciones críticas como la dificultad de encontrar estacionamiento, especialmente en áreas muy densamente pobladas como Manhattan. Los conductores de reparto, como son los de UPS, se enfrentan a largas esperas para estacionarse, lo que ralentiza las entregas y aumenta los costos de operación. Como respuesta, empresas y ciudades en Europa y Asia han hecho esfuerzos para implementar medidas innovadoras, como horarios restringidos para entregas, centros logísticos centralizados y vehículos ecológicos, para de esta manera mejorar la eficiencia. De manera adicional, se exploran en estos momentos tecnologías emergentes como drones, robots de reparto y vehículos autónomos para optimizar las entregas y reducir los costos.

Para este proyecto buscamos abordar los desafíos que se presentan mediante la implementación de un algoritmo de colonia de hormigas para resolver el problema del viajero dadas un conjunto de coordenadas geográficas, optimizando así las rutas de entrega en escenarios urbanos complejos. De igual forma, se utiliza Python para permitirnos explorar soluciones eficientes para minimizar los tiempos de entrega y al mismo tiempo maximizar la eficiencia de operación.


# **Importación de Librerias**

1. `Numpy`: Biblioteca utilizada fundamentalmente para el cálculo numérico en Python, en particular es usada para poder trabajar con matrices o matrices multidimencionales.

2. `Pandas`: Biblioteca es usada para la manipulación y análisis de datos estructurados, especialmente con un formato tabular. Por otro lado, esta permite importar datos en formatos CSV, Excel, SQL, entre otros.

3. `Plotly`: Biblioteca la cual se usa para la creación de gráficos interactivos y visualizaciones de datos, dando a los gráficos una autenticidad útil al momento de presentar información.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# **Funciones**

## **Función para calcular la distancia euclidiana**

La función `distancia` tiene como objetivo realizar el cálculo entre la distancia de dos puntos ubicados cartesianamente, se escogió la distancia euclidiana, que funciona calculando la diferencia de las componentes en latitud y longitud. utilizando la siguiente fórmula.

<center>$Distancia = \sqrt{(x_1-x_2)^2+(y_1-y_2)^2}$</center>

* **input**: Recibe como parametro `c1` y `c2` los cuales representan a ciudades en coordenadas especificas.
* **output**: Retorna la sitancia euclidiana entre `c1` y `c2`.

In [2]:
# Función de distancia: calcula la distancia euclidiana entre dos puntos
def distancia(c1, c2):
    return np.linalg.norm(np.array(c1) - np.array(c2))

## **Creación de la matriz de distancias entre las 72 ciudades**

La función `inicializar_matriz_distancias` realizara una matriz de distancias, es necesario resaltar que la matriz que se producirá será simétrica, esto debido a que la distancia entre una ciudad $“i”$ a una ciudad $“j”$ es la misma que la distancia entre la ciudad $“j”$ a la ciudad $“i”$. Inicialmente la función creara una matriz de tamaño ($n$ x $n$) unicamente con zeros, donde n es la cantidad de longitud de `ciudades`. Posteriormente, se realiza un bucle a lo largo de las longitudes y latitudes para asignar las distancias utilizando la función `distancia (c1,c2)`.

* **input**: Recibe como parametro `ciudades` la cual es un arreglo con las coordenadas de todas las ciudades.
* **output**: Matriz con todas las distancias entre cada una de las ciudades.

In [3]:
# Inicializa la matriz de distancias
def inicializar_matriz_distancias(ciudades):
    n = len(ciudades)
    distancias = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
          dis = distancia(ciudades[i], ciudades[j])
          distancias[i, j] = dis
          distancias[j, i] = dis
    return distancias

## **Inicialización de feromonas**

Esta función `inicializar_feromonas` tiene como objetivo inicializar una matriz de feromonas, si no se especifica un valor de `valor_inicial`, se le dará por defecto un valor de 1.

* **input**: Recibe como parametro `n_ciudades` siendo esto el numero de ciudades en el modelo, y tambien recibe `valor_inicial` que sera el numero de feromonas con el que se inicializaran los caminos entre ciudades del modelo.
* **output**: Matriz con con el valor inicial de las feromonas en el modelo.

In [4]:
# Inicializa la matriz de feromonas
def inicializar_feromonas(n_ciudades, valor_inicial=1.0):
    return np.ones((n_ciudades, n_ciudades)) * valor_inicial

## **Función para elegir la siguiente ciudad a dirigirse**

La función `elegir_ciudad` está diseñada para elegir a partir de probabilidades la siguiente ciudad a visitar, dicha elección se realiza basándose en dos factores, las feromonas y las distancias entre las ciudades. Adicionalmente, los parámetros alfa y beta dan mayor peso a estos dos factores.

Al inicio se calcula `tau`, vector de las feromonas de las ciudades que aún no se visitan y tambien se calcula `eta` el inverso de las distancias entre la ciudad actual y las ciudades faltantes, esto debido a que el objetivo es minimizar la distancia recorrida, por lo tanto, cuanto mayor sea el valor de distancia inversa, mayor probabilidad de ser seleccionadas. .

Posteriormente se calculan las probabilidades utilizando los valores de `tau` y `eta` con los valores de `alfa` y `beta` para así calcular las probabilidades de ir a cada ciudad, teniendo mayor probabilidad de ser elegidas las ciudades con mas feromonas y menos distanca a la ciudad en la que se encuntra la hormiga.

<center>  <p>
$$
P_{ij} = \frac{[\tau_{ij}]^\alpha \cdot [\eta_{ij}]^\beta}{\sum_{k \in J}[\tau_{ik}]^\alpha \cdot [\eta_{ik}]^\beta}
$$

  </p></center>


* **input**: Recibe como parametros `feromonas`,`distancias`,`ciudad_0`,`ciudades_faltantes`,`alfa` y `beta` siendo estos:
    *  `feromonas`: cantidad de feromonas entre la ciudad en la que se encuntra la hormiga y las que aún no ha visitado.
    *  `distancias`: distancia  entre la ciudad en la que se encuntra la hormiga y las que aún no ha visitado.
    *  `ciudad_0`: ciudad en la que se encuntra la hormiga.
    *  `ciudades_faltantes`: ciudades que aún no visita la hormiga.
    *  `alfa`: Importancia que se le dara a las feromonas para determinar la proxima ciudad a visitar.
    *  `beta`:Importancia que se le dara a la distancia para determinar la proxima ciudad a visitar.


* **output**: Proxima ciudad a la que ira la hormiga.

In [5]:
def elegir_ciudad(feromonas, distancias, ciudad_0, ciudades_faltantes, alfa, beta):
    # Accede solo una vez a las matrices para las ciudades faltantes
    tau = feromonas[ciudad_0, ciudades_faltantes]  # Vector de feromonas para las ciudades no visitadas
    eta = 1 / distancias[ciudad_0, ciudades_faltantes]  # Vector de inversa de distancias

    # Calcula los valores de probabilidad utilizando operaciones vectorizadas
    p_values = (tau ** alfa) * (eta ** beta)

    # Normaliza las probabilidades
    p_values /= p_values.sum()

    # Selecciona la ciudad de forma probabilística
    ciudad_siguiente = np.random.choice(ciudades_faltantes, p=p_values)
    return ciudad_siguiente

## **Ejecución del Algoritmo de Colonia de Hormigas**

La función `aco_tsp` tiene como proposito ser la funcion principal del algoritmo en donde se buscara la ruta optima para el problema del viajero `tsp`.

Para iniciar el código, se calcula el número de ciudades, posteriormente se genera la matriz de distancias, con la funcion `inicializar_matriz_distancias`, y se inicializan las feromonas, con la función `inicializar_feromonas`.

En el primer ciclo se inicia con la creación de una lista que almacenará las rutas encontradas por cada hormiga, además se generará una variable que almacena la distancia de cada ruta de hormigas.

Mientras que en el segundo ciclo se construye una ruta por cada hormiga, donde se elige aleatoriamente una ciudad inicial y se comienza una ruta con esa ciudad, aparte se crea un conjunto de ciudades que aún no son visitadas.

En este proceso se encuntra la selección de la siguiente ciudad a visitar, con la función `elegir_ciudad`,  donde mientras existan ciudades sin visitar,y no se haya pasado el numero maximo de iteraciones, la hormiga seleccionará una ciudad, después de seleccionar la ciudad se agrega a la ruta y se elimina del conjunto de ciudades no visitadas.

Una vez que todas las ciudades han sido visitadas, la hormiga vuelve a la ciudad de origen completando así su ruta, para que posteriormente se almacene en la lista de rutas de hormigas.

Posteriormente se calcula la distancia total recorrida por la hormiga, si esta distancia es mejor que la distancia global conocida, se actualiza la mejor distancia al igual que la mejor ruta.

Luego de la iteración principal se aplica la actualización a la matriz de feromonas y se genera un ciclo adicional donde para cada hormiga se calcula el valor de feromonas depositadas en función de la calidad de la ruta, donde la cantidad de feromonas es inversamente proporcional a la distancia de la ruta. Finalmente se actualiza la matriz de feromonas por cada par de ciudades en la ruta de la hormiga, para si al final de la función regresar la mejor ruta y la mejor distancia.

* **input**: Recibe como parametros `ciudades`,`n_hormigas`,`alfa`,`beta`,`rho`,`q` y `iteraciones` siendo estos:

    *  `ciudades`: lista con las coordenadas de todas las ciudades que se visitaran en el problema.
    *  `n_hormigas`: numero de hormigas que estaran en el modelo.
    *  `alfa`: Importancia que se le dara a las feromonas para determinar la proxima ciudad a visitar. Por defecto 1.
    *  `beta`:Importancia que se le dara a la distancia para determinar la proxima ciudad a visitar. Por defecto 2.
    *  `rho`: Coeficiente de evaporación de feromonas. Por defecto 0.5.
    *  `q`: Cantidad de feromonas que dejara una hormiga. Por defecto 100.
    *  `iteraciones`: Numero de iteraciones que hara el modelo antes de calcular la mejor distancia. Por defecto 100.


* **output**: `mejor_ruta_global` y `mejor_distancia_global`, donde la primera es una lista con el orden en el que fue visitada cada ciudad en la solución optima y el segundo la distancia que reccore la solución optima encontrada.

In [6]:
def aco_tsp(ciudades, n_hormigas, alfa=1, beta=2, rho=0.5, q=100, iteraciones=100):
    n_ciudades = len(ciudades)
    distancias = inicializar_matriz_distancias(ciudades)
    feromonas = inicializar_feromonas(n_ciudades)

    mejor_distancia_global = np.inf
    mejor_ruta_global = None

    for iteracion in range(iteraciones):
        rutas_hormigas = []
        distancias_rutas = np.zeros(n_hormigas)

        # Cada hormiga construye una ruta
        for hormiga in range(n_hormigas):
            ciudad_inicial = np.random.randint(0, n_ciudades)
            ruta = [ciudad_inicial]
            ciudades_no_visitadas = set(range(n_ciudades))
            ciudades_no_visitadas.remove(ciudad_inicial)

            while ciudades_no_visitadas:
                ciudad_actual = ruta[-1]
                ciudad_siguiente = elegir_ciudad(
                    feromonas, distancias, ciudad_actual, list(ciudades_no_visitadas), alfa, beta
                )
                ruta.append(ciudad_siguiente)
                ciudades_no_visitadas.remove(ciudad_siguiente)

            # Vuelve a la ciudad inicial para completar la ruta
            ruta.append(ciudad_inicial)
            rutas_hormigas.append(ruta)

            # Calcula la distancia de la ruta de manera vectorizada
            distancias_rutas[hormiga] = np.sum([distancias[ruta[i], ruta[i + 1]] for i in range(n_ciudades)])

            # Actualiza la mejor ruta global si se encuentra una mejor
            if distancias_rutas[hormiga] < mejor_distancia_global:
                mejor_distancia_global = distancias_rutas[hormiga]
                mejor_ruta_global = ruta

        # Actualización de feromonas: evaporación (se hace vectorizado)
        feromonas *= (1 - rho)

        # Depósito de feromonas por las hormigas
        for hormiga, ruta in enumerate(rutas_hormigas):
            delta_feromona = q / distancias_rutas[hormiga]
            for j in range(n_ciudades):
                i, k = ruta[j], ruta[j + 1]
                feromonas[i, k] += delta_feromona
                feromonas[k, i] += delta_feromona

    return mejor_ruta_global, mejor_distancia_global

## **Producción de gráfica**

Esta función tiene como objetivo producir el cambio de color de manera moderada entre el azul que se establece como el punto de inicio de la ruta hasta el verde que se establece como el punto final de la ruta, esto con el objetivo de diferenciar la dirección que tomó la hormiga que produjo la mejor ruta con la mejor distancia, asi plazmandolo en el grafico.

Esta función toma como parametro de entrada el formato de color RGB y lo pasa a valor hexadecimal.

Como última función se tiene la graficación de la ruta, esta toma en cuenta lista de ciudades y la ruta final encontrada por el algoritmo, adicionalmente toma en cuenta el color de inicio y de final de la ruta y se establecen los valores del eje `x` y del eje `y`.

Luego se empieza a graficar la conexión entre las ciudades con los colores interpolados, utilizando las funciones de interpolación de color y conversor de color a hexadecimal.

Finalmente se marca cual es la ciudad de inicio con el color morado para que se resalte y se note cual es, además de agregar un personalizado oscuro a la gráfica y asignando los nombres de los ejes y leyendas respectivamente.

Por último, se asigna la posición de la leyenda hasta abajo del gráfico y se manda la instrucción de imprimir la gráfica.

In [7]:
# Función para interpolar colores en un gradiente de morado a azul claro
def interpolar_color(color_inicio, color_fin, factor):
    return tuple([
        int(color_inicio[i] + (color_fin[i] - color_inicio[i]) * factor)
        for i in range(3)
    ])

# Convertir un valor RGB a hexadecimal
def rgb_a_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

def graficar_ruta(ciudades, ruta, titulo="Ruta óptima"):
    ciudades = np.array(ciudades)
    ruta = np.array(ruta)

    # Colores de inicio (morado) y fin (azul claro)
    color_inicio = (0, 105, 148)  # AguaMarina (en RGB)
    color_fin = (144, 238, 144)  # Verde Claro (en RGB)

    # Crear una figura en Plotly
    fig = go.Figure()

    # Graficar las conexiones entre ciudades en la ruta con colores interpolados
    num_segmentos = len(ruta) - 1
    for i in range(num_segmentos):
        ciudad_actual = ruta[i]
        ciudad_siguiente = ruta[i + 1]

        # Interpolar el color para este segmento
        factor = i / num_segmentos  # Proporción de progreso en la ruta
        color_interpolado = interpolar_color(color_inicio, color_fin, factor)
        color_hex = rgb_a_hex(color_interpolado)

        # Graficar el segmento de la ruta con el color interpolado
        fig.add_trace(go.Scatter(
            x=[ciudades[ciudad_actual, 0], ciudades[ciudad_siguiente, 0]],
            y=[ciudades[ciudad_actual, 1], ciudades[ciudad_siguiente, 1]],
            mode='lines+markers',
            line=dict(color=color_hex, width=2),
            marker=dict(size=8, color=color_hex),  # Los puntos ahora toman el color interpolado
            showlegend=False
        ))

        # Graficar cada ciudad de la ruta con el color interpolado
        fig.add_trace(go.Scatter(
            x=[ciudades[ciudad_actual, 0]],
            y=[ciudades[ciudad_actual, 1]],
            mode='markers',
            marker=dict(size=10, color=color_hex),  # Puntos con el color interpolado
            showlegend=False
        ))

    # Marcar la ciudad de inicio/final con un color distinto
    fig.add_trace(go.Scatter(
        x=[ciudades[ruta[0], 0]], y=[ciudades[ruta[0], 1]],
        mode='markers',
        marker=dict(size=12, color='lightgreen'),
        name='Inicio/Final'
    ))
    # Añadir una escala de colores como barra de color (colorbar)
    fig.add_trace(go.Scatter(
        x=[None], y=[None],  # Elemento invisible
        mode='markers',
        marker=dict(
            colorscale=[[0, rgb_a_hex(color_inicio)], [1, rgb_a_hex(color_fin)]],
            cmin=0, cmax=1,
            colorbar=dict(
                title="Progreso en la ruta",  # Título de la barra de colores
                titleside="right",
                tickvals=[0, 1],
                ticktext=["Inicio", "Final"],
            ),
            showscale=True
        ),
        hoverinfo='none',
        showlegend=False
    ))

    # Formato de la gráfica personalizado
    fig.update_layout(
        title={'text': f'<b>{titulo}</b>', 'x': 0.5, 'xanchor': 'center'},  # Centrar título y formato en negrita
        xaxis_title='Coordenada X',
        yaxis_title='Coordenada Y',
        template='plotly_dark',  # Tema oscuro
        showlegend=True,

        # Ejes en negrita
        xaxis=dict(
            title='Longitud',
            titlefont=dict(size=14),
            tickfont=dict(size=12, family='Arial', color='white'),
        ),
        yaxis=dict(
            title='Latitud',
            titlefont=dict(size=14),
            tickfont=dict(size=12, family='Arial', color='white'),
        ),

        # Leyenda en la parte inferior
        legend=dict(
            orientation="h",  # Horizontal
            yanchor="bottom", y=-0.3,
            xanchor="center", x=0.5
        )
    )

    # Mostrar la gráfica
    fig.show()


# **Generación del Problema y Graficación**

 Se lee el csv donde se guarda la información y se guarda como “Coordenadas_reto.csv”, posteriormente se aplicar el **Algoritmo de Colonia de Hormigas** y finalmente se grafica la mejor ruta obtenidad.

In [10]:
#CSV con coordenadas
Coordenadas = pd.read_csv('./Coordenadas_reto.csv').dropna().drop_duplicates()

#Establecer ciudades
ciudades = [(x, y) for x, y in zip(Coordenadas.iloc[:, 0], Coordenadas.iloc[:, 1])]

#Ejecución del Problema
mejor_ruta, mejor_distancia = aco_tsp(ciudades, n_hormigas=72, iteraciones=72)

print(f"Mejor ruta: {mejor_ruta}")
print(f"Mejor distancia: {mejor_distancia}")
graficar_ruta(ciudades, mejor_ruta, titulo="Ruta óptima con ACO")

Mejor ruta: [8, np.int64(7), np.int64(9), np.int64(5), np.int64(17), np.int64(4), np.int64(48), np.int64(41), np.int64(42), np.int64(38), np.int64(44), np.int64(2), np.int64(3), np.int64(47), np.int64(37), np.int64(39), np.int64(46), np.int64(49), np.int64(0), np.int64(19), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(28), np.int64(15), np.int64(14), np.int64(13), np.int64(12), np.int64(6), np.int64(32), np.int64(43), np.int64(45), np.int64(1), np.int64(10), np.int64(20), np.int64(18), np.int64(16), np.int64(64), np.int64(63), np.int64(62), np.int64(61), np.int64(60), np.int64(70), np.int64(69), np.int64(65), np.int64(66), np.int64(67), np.int64(68), np.int64(59), np.int64(58), np.int64(57), np.int64(55), np.int64(56), np.int64(53), np.int64(50), np.int64(54), np.int64(52), np.int64(51), np.int64(27), np.int64(26), np.int64(30), np.int64(29), np.int64(31), np.int64(33), np.int64(36), np.int64(35), np.int64(34), np.int64(40), np.int64(11), 8]
Mejor dist

# **Variación de parametros**

Se variaran los parametros `ciudades`,`n_hormigas`,`alfa`,`beta`,`rho`,`q` y `iteraciones`de la función `aco_tsp`, para poder ver si la modificación de uno de estos parametros encontra una solución optima.

**Variación de alfa**:


*   1: Valor por defecto de alfa.
*   2: Igualar la importancia de las feromonas y la distancia para escoger un camino.
*   3: Aumentar la importancia de las feromonas respecto a la distancia para elegir un camino.



In [11]:
# Variación de alfa = {1,2,3}
for i in range(1,4):
  print(f"Alfa = {i}")
  %time _, mejor_distancia = aco_tsp(ciudades, n_hormigas=72, iteraciones=72, alfa = i)
  print(f"Mejor distancia: {mejor_distancia}")
  print("---------------------------------------------------------------------------------------------")

Alfa = 1
CPU times: total: 8.88 s
Wall time: 8.88 s
Mejor distancia: 1716.9124537799994
---------------------------------------------------------------------------------------------
Alfa = 2
CPU times: total: 8.88 s
Wall time: 8.86 s
Mejor distancia: 1707.6764761485008
---------------------------------------------------------------------------------------------
Alfa = 3
CPU times: total: 9.05 s
Wall time: 9.14 s
Mejor distancia: 1774.6827602281853
---------------------------------------------------------------------------------------------


**Variación de beta**:


*   1: Valor por defecto de beta.
*   2: Aumentar la importancia de la distancia respecto a las feromonas para elegir un camino.
*   3: Aumentar de mayor manera la importancia de la distancia respecto a las feromonas para elegir un camino.

In [ ]:
# Variación de beta = {2,3,4}
for i in range(2,5):
  print(f"Beta = {i}")
  %time _, mejor_distancia = aco_tsp(ciudades, n_hormigas=72, iteraciones=72, beta = i)
  print(f"Mejor distancia: {mejor_distancia}")
  print("---------------------------------------------------------------------------------------------")

Beta = 2
CPU times: user 21.8 s, sys: 532 ms, total: 22.4 s
Wall time: 22.2 s
Mejor distancia: 1695.6251536847303
---------------------------------------------------------------------------------------------
Beta = 3
CPU times: user 21.9 s, sys: 469 ms, total: 22.4 s
Wall time: 22.2 s
Mejor distancia: 1671.9776241541538
---------------------------------------------------------------------------------------------
Beta = 4
CPU times: user 22.6 s, sys: 511 ms, total: 23.1 s
Wall time: 22.8 s
Mejor distancia: 1646.5273222051092
---------------------------------------------------------------------------------------------


**Variación de rho**:


*   0.25: Disminuir a un `25%` la tasa de evaporación de feromonas, lo que hara que haya más feromonas en los caminos.
*   0.5: Valor por defecto de rho.
*   0.75: Aumentar a un `75%` la tasa de evaporación de feromonas, lo que hara que haya menos feromonas en los caminos.

In [ ]:
# Variación de rho = {0.25,0.5,0.7}
for i in [0.25,0.5,0.75]:
  print(f"Rho = {i}")
  %time _, mejor_distancia = aco_tsp(ciudades, n_hormigas=72, iteraciones=72, rho = i)
  print(f"Mejor distancia: {mejor_distancia}")
  print("---------------------------------------------------------------------------------------------")

Rho = 0.25
CPU times: user 20.7 s, sys: 233 ms, total: 20.9 s
Wall time: 20.9 s
Mejor distancia: 1725.0073171917393
---------------------------------------------------------------------------------------------
Rho = 0.5
CPU times: user 20 s, sys: 195 ms, total: 20.2 s
Wall time: 20.2 s
Mejor distancia: 1657.278692436558
---------------------------------------------------------------------------------------------
Rho = 0.75
CPU times: user 20.2 s, sys: 239 ms, total: 20.4 s
Wall time: 20.4 s
Mejor distancia: 1715.4393367923371
---------------------------------------------------------------------------------------------


**Variación de q**:


*   50: Disminuir la cantidad de feromonas que deja cada hormiga, lo que hara que haya más feromonas en los caminos.
*   100: Valor por defecto de q.
*   200: Aumentar la cantidad de feromonas que deja cada hormiga, lo que hara que haya menos feromonas en los caminos.

In [ ]:
# Variación de q = {50,100,200}
for i in [50,100,200]:
  print(f"q = {i}")
  %time _, mejor_distancia = aco_tsp(ciudades, n_hormigas=72, iteraciones=72, q = i)
  print(f"Mejor distancia: {mejor_distancia}")
  print("---------------------------------------------------------------------------------------------")

q = 50
CPU times: user 20.5 s, sys: 197 ms, total: 20.7 s
Wall time: 20.7 s
Mejor distancia: 1764.074559469537
---------------------------------------------------------------------------------------------
q = 100
CPU times: user 19.5 s, sys: 168 ms, total: 19.7 s
Wall time: 19.6 s
Mejor distancia: 1707.872829419575
---------------------------------------------------------------------------------------------
q = 200
CPU times: user 20.6 s, sys: 196 ms, total: 20.8 s
Wall time: 20.8 s
Mejor distancia: 1663.5370486593774
---------------------------------------------------------------------------------------------
